# <font color=#000000, style="background-color:Gray;">__[TODO]__</font>

1) Move [DBG] calls out of <font color=#000000, style="background-color:Yellow;">__MODULES AND FUNCTIONS__</font> and into <font color=#000000, style="background-color:Yellow;">__TESTING__</font>
2) Find and perform any improvements to code logic, style, organization (e.g. simpler logic, refactoring, etc.)
3) Confirm if $\texttt{results[i]}$ is computed properly in $\texttt{\textcolor{CornflowerBlue}{\_detection\_results}(...)}$
4) Clean up both calls to $\texttt{exit()}$ in $\texttt{\textcolor{CornflowerBlue}{\_update\_indices}(...)}$
5) Fix how $\texttt{indices\_err2}$ is printed by $\texttt{DBG()}$ call at the end of $\texttt{\textcolor{CornflowerBlue}{\_update\_indices}(...)}$
6) <font color=yellow>__COMPLETE__</font> $\texttt{\textcolor{CornflowerBlue}{\_init\_2nd\_check}(...)}$ so that it actually checks if_error (and rename it accordingly)
7) Check that the $\texttt{bases[i] == "iY"}$ case is using the correct operator in $\texttt{\textcolor{CornflowerBlue}{\_init\_2nd\_check}(...)}$ so that it actually checks if_error (and rename it accordingly)
8) Confirm if phase_data encoding is using the correct operator in $\texttt{\textcolor{CornflowerBlue}{\_encode\_data}(...)}$
10) Complete code for Step 5
11) Complete code for Step 6
12) Refactor everything in <font color=#000000, style="background-color:Yellow;">__TESTING__</font> into a $\texttt{qisac\_protocol(...)}$ function
13) Find and perform any improvements to code logic, style, organization (e.g. simpler logic, refactoring, etc.)
14) Perform comprehensive unit tests

# <font color=#000000, style="background-color:Yellow;">__MODULES AND FUNCTIONS__</font>

## Modules

In [1]:
# UI and debugging
from termcolor import colored

# Protocol
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit import QuantumRegister, ClassicalRegister
from math import ceil
import numpy as np
import random
import math


# Visualization
from qiskit.visualization import plot_circuit_layout, circuit_drawer, plot_bloch_multivector
from pylatexenc.latexencode import unicode_to_latex
from qiskit.quantum_info import Statevector

## Debug Functions

In [2]:
# Prints debug message (if _dbg_ == mode)
def DBG(msg, mode):
    global _dbg_
    if _dbg_ >= mode:
        print("\n" + colored("[DBG]", "black", "on_yellow", attrs=["bold"]) + "\t" + msg)


# Like DBG, but without icon
def LST(msg, mode):
    global _dbg_
    if _dbg_ >= mode:
        print("\n\t" +  msg)
            

# Prints error message if is_error == True; print success message otherwise
def ERR(msg_err, msg_suc, is_error):
    global _dbg_
    if _dbg_:
        if is_error:
            print("\n" + colored("[ERR]", "black", "on_red", attrs=["bold", "blink"]) + "\t" +  msg_err)
            #raise Exception("User Defined Error")
        else:
            print("\n" + colored("[ "+u'\u2713'+" ]", "black", "on_green", attrs=["bold", "blink"]) + "\t" +  msg_suc)


# New task
def NEW(msg):
    print("\n\n\n" + colored("[NEW]", "black", "on_cyan", attrs=["bold"]) + "\t" + msg)


# Used for uninitialized variables
def NIL(var="variable"):  # var is name of unitialized variable
    ERR(var + " is unitialized! Exiting...", "", True) 

## Functions

In [3]:
# Step 1.1 (compute numbers of EPR pairs needed for message, eavesdrop checks, and total)
def get_EPR_counts(message, pe, m, m_msg, m_err1, m_err2):
    # first eavesdrop pairs
    for m in range(len(message)*2):
        m_err1 = ceil((1-pe)/2*m)
        if m - m_err1 >= len(message) and (m - m_err1)%2 == 0:
            m_msg = m
            break
    m_msg = m - m_err1
    # second eavesdrop pairs
    for m in range((len(message))*2):
        m_err2 = ceil((1-pe)/2*m)
        if m - (m_err1+m_err2) >= len(message) and (m - m_err2)%2 == 0:
            m_msg = m
            break
    m_msg = m - m_err1 - m_err2
    # return counts
    DBG("m      = " + str(m) + "\n\tm_msg  = " + str(m_msg) + "\n\tm_err1 = " + str(m_err1) + "\n\tm_err2 = " + str(m_err2), 2)
    return m, m_msg, m_err1, m_err2
    

# Step 1.2 (create EPR pairs)
def create_EPR_pairs(m):
    EPR_pairs = []
    for i in range(m):
        label_A          = r"""A_"""+str(i)
        label_B          = r"""B_"""+str(i)
        q_a              = QuantumRegister(1, name=label_A)
        q_b              = QuantumRegister(1, name=label_B)
        c                = ClassicalRegister(2)
        EPR_pair_circuit = QuantumCircuit(q_a, q_b, c)
        EPR_pair_circuit.x(q_b[0])
        EPR_pair_circuit.h(q_a[0])
        EPR_pair_circuit.z(q_a[0])
        EPR_pair_circuit.z(q_b[0])
        EPR_pair_circuit.cx(q_a[0], q_b[0])
        EPR_pairs.append(EPR_pair_circuit)
    DBG(str(len(EPR_pairs)) + " EPR pairs generated", 1)
    return EPR_pairs


# Step 2
def detect_eavesdropping(simulator, _circuits, m, m_err1, indices_err1, pe, threshold, debug_value = -1):   # [DBG] debug_value == True or debug_value == False
    # Randomly select eavesdropping pairs
    num_samples = m_err1
    indices_err1 = np.random.choice(m, num_samples, replace=False)
    if   debug_value == False:
        return False
    elif debug_value == True:
        return True
    else:
        # Get Bob's basis selections
        bases = {}
        for i in indices_err1:
            bases[i] = random.choice(['X', 'Y', 'Z'])
        # Get Alice and Bob's results
        bob_results   = _detection_results(simulator, _circuits, indices_err1, bases, 1)  # 1 is Bob's qubit
        alice_results = _detection_results(simulator, _circuits, indices_err1, bases, 0)  # 0 is Alice's qubit
        # Count errors
        error_count = 0
        DBG("Eavesdrop detection circuits:", 3)
        for i in indices_err1:
            LST("\n" + str(_circuits[i]) + "\n", 3)
            if alice_results[i] == bob_results[i]:
                error_count += 1
        DBG("bob_results       = " + str(bob_results) + "\n\talice_results     = " + str(alice_results), 2)
        DBG("error_count       = "+str(error_count) + "\n\tlen(indices_err1) = "+str(len(indices_err1)) + "\n\tthreshold         = "+str(threshold),2)
        is_error = (error_count / len(indices_err1)) >= threshold
        ERR("Eavesdropping (1) detected! Restarting QISAC protocol...", "No eavesdropping detected. Continuing...", is_error)
    if not is_error:
        # Remove eavesdropping detection pairs from circuits and return error status
        DBG("Removing eavesdrop detection circuits from circuits list", 2) 
        indices_err1 = sorted(indices_err1, reverse=True)
        for i in indices_err1:
            _circuits.pop(i)
    return _circuits, indices_err1, is_error


# Step 3:
def encode(circuits, message, phase_data, m_err2, indices_msg, indices_err2):
    # Check length restrictions and abord or add padding as needed
    if len(message) > len(circuits) - m_err2:
        ERR("message / phase_data lengths cannot exceed available qubits! Exiting...", "", True)
    elif len(message) < len(circuits) -  m_err2:
        for i in range(len(circuits) - len(message) - m_err2):
            message    = "".join([message, "0"])     # [NOTE] we assume message cannot end with a 0 
            phase_data = "".join([phase_data, "0"])  # [NOTE] we assume phase_data cannot end with a 0 
        DBG("Padded message    = " + message + "\n\tPadded phase_data = " + phase_data, 1)
    indices_msg, indices_err2 = _update_indices(message, phase_data, indices_msg, indices_err2)
    circuits, bases           = _init_2nd_check(circuits, indices_err2)
    circuits                  = _encode_data(circuits, message, phase_data, indices_msg)    
    return circuits, message, phase_data, indices_msg, indices_err2, bases

# Step 4:
def detect_eavesdropping_2(simulator, _circuits, m_msg, m_err2, indices_err2, bases_err2, pe, threshold):
    # Get Alice and Bob's results
    bob_results   = _detection_results(simulator, _circuits, indices_err2, bases_err2, 1)  # 1 is Bob's qubit
    alice_results = _detection_results(simulator, _circuits, indices_err2, bases_err2, 0)  # 0 is Alice's qubit
    # Count errors
    error_count = 0
    DBG("Eavesdrop detection circuits:", 3)
    for i in indices_err2:
        LST("\n" + str(_circuits[i]) + "\n", 3)
        if alice_results[i] == bob_results[i]:
            error_count += 1
    DBG("bob_results       = " + str(bob_results) + "\n\talice_results     = " + str(alice_results), 2)
    DBG("error_count       = "+str(error_count) + "\n\tlen(indices_err2) = "+str(len(indices_err2)) + "\n\tthreshold         = "+str(threshold),2)
    is_error = (error_count / len(indices_err2)) >= threshold
    ERR("Eavesdropping (2) detected! Restarting QISAC protocol...", "No eavesdropping detected. Continuing...", is_error)
    if not is_error:
        # Remove eavesdropping detection pairs from circuits and return error status
        DBG("Removing eavesdrop detection circuits from circuits list", 2) 
        indices_err2 = sorted(indices_err2, reverse=True)
        for i in indices_err2:
            _circuits.pop(i)
    
    return _circuits, is_error

## Utility Functions

In [4]:
# Helper function for Step 2
def _detection_results(simulator, circuits, indices, bases, qubit):
    results = {}
    for i in indices:
        # Prepare circuits at indices
        if   bases[i] == 'X':
            circuits[i].h(qubit)
        elif bases[i] == 'Y':
            circuits[i].sdg(qubit)
            circuits[i].h(qubit)
        # Get results
        circuits[i].measure(qubit, qubit)
        circuits[i] = transpile(circuits[i], simulator)
        job         = simulator.run(circuits[i], shots=1)
        result      = job.result().get_counts()
        result_str  = list(result.keys())[0]
        results[i]  = result_str[qubit]  # [TODO] Confirm this is correct
    return results


# Helper function for Step 3
def _update_indices(message, phase_data, indices_msg, indices_err2):
    if len(message) != len(phase_data):
        ERR("len(message) != len(phase_data)", "", True)
        exit()  # [TODO] clean this up
    indices_err2 = np.random.choice(m, m_err2, replace=False)
    # Get circuits indices left for message
    indices_msg = []
    for i in range(len(circuits)):
        indices_msg.append(i)
    for i in sorted(range(len(circuits)), reverse=True):
        for j in indices_err2:
            if i == j:
                indices_msg.pop(j)
    if len(indices_msg) != len(message):
        ERR("len(indices_msg) != len(message)", "", True)
        exit()  # [TODO] clean this up
    DBG("indices_msg    = " + str(indices_msg) + "\n\tindices_err2   = " + str(indices_err2), 2)  # [TODO] Fix print of indices_err2
    return indices_msg, indices_err2


# Helper function for Step 3
def _init_2nd_check(circuits, indices_err2):
    bases = {}
    for i in indices_err2:
        bases[i] = random.choice(["I", "Z", "X", "iY"])
        if   bases[i] == "Z":
            circuits[i].z(0)
        elif bases[i] == "X":
            circuits[i].x(0)
        elif bases[i] == "iY":
            circuits[i].ry(np.pi/2, 0)  # [TODO] Is this the correct operator?
    DBG("bases          =  " + str(bases), 2)
    return circuits, bases

# Helper function for Step 3
def _encode_data(circuits, message, phase_data, indices_msg):
    # Encode message
    DBG("Encoding message", 1)
    j = 0
    for i in indices_msg:
        if message[j] == "1":
            circuits[i].x(0)
        j += 1
    # Encode phase_data
    for i in indices_msg:
        circuits[i].rz(math.radians(int(phase_data)), 0) # [TODO] Is this the correct operator?
    DBG("Encoded message/parameter and 2nd eavesdrop check circuits:", 3)
    for c in circuits:
        LST("\n" + str(c), 3)
    return circuits

# <font color=#000000, style="background-color:Yellow;">__QISAC TESTING__</font>

<font color=#000000, style="background-color:Yellow;">__[ ! ]__</font> $\hspace{0.2mm}$ Code in this section can be rolled into a $\texttt{qisac()}$ function with all global variables as parameters for "production" usage

In [5]:
# Global variable declarations and initializations
_dbg_        = 2                    # If _dbg_, then print additional debug statements
m            = None                 # Number of EPR pairs
m_msg        = None                 # Number of EPR pairs allocated to message / phase accumulation
m_err1       = None                 # Number of EPR pairs allocated to first eavesdropping check
m_err2       = None                 # Number of EPR pairs allocated to second eavesdropping check
indices_msg  = None                 # Indices from circuits allocated to message / phase accumulation
indices_err1 = None                 # Indices from circuits allocated to first eavesdropping check (won't be meaningful after Step 2)
indices_err2 = None                 # Indices from circuits allocated to second eavesdropping check
pe           = 0.8                  # Proportion of EPR pairs NOT used for 1st error detection
threshold    = 0.5                  # Threshold for first eavesdropping detection
simulator    = AerSimulator()       # Create simulator
circuits     = None                 # Used to store list of EPR pairs
is_error     = True                 # Used to indicate if eavesdropping was detected (see Step 2)
message      = "01110111"           # Message to be communicated
phase_data   = "10110111"           # Phase data to be accumulated
N            = 1                    # Number of evolution iterations of phase accumulation
bases_err2   = None                 # Bases selected by Alice for 2nd eavesdropping check


while is_error:

    print("\n" + colored("[STARTING QISAC PROTOCOL]", "black", "on_magenta", attrs=["bold"]))
    
    NEW("(1) Prepare list of EPR pairs and send one qubit from each to Bob")
    m, m_msg, m_err1, m_err2 = get_EPR_counts(message, pe, m, m_msg, m_err1, m_err2)
    circuits                 = create_EPR_pairs(m)  # Create list of EPR pair circuits

    NEW("(2) Check for eavesdropping")
    # add extra param to detect_eavesdropping() as either True or False to debug
    circuits, indices_err1, is_error = detect_eavesdropping(simulator, circuits, m, m_err1, indices_err1, pe, threshold)  

    if is_error:
        print("\n")  # UI formatting
        continue

    NEW("(3) Encode message and parameter estimate")
    circuits, message, phase_data, indices_msg, indices_err2, bases_err2 = encode(circuits, message, phase_data, m_err2, indices_msg, indices_err2)


    NEW("(4) Transmit remaining qubits and check again for eavesdropping")
    cicruits, is_error = detect_eavesdropping_2(simulator, circuits, m, m_err2, indices_err2, bases_err2, pe, threshold)

    if is_error:
        print("\n")  # UI formatting
        continue

NEW("(5) Perform measurements")


NEW("(6) Estimate parameter")



[STARTING QISAC PROTOCOL]



[NEW]	(1) Prepare list of EPR pairs and send one qubit from each to Bob

[DBG]	m      = 12
	m_msg  = 9
	m_err1 = 1
	m_err2 = 2

[DBG]	12 EPR pairs generated



[NEW]	(2) Check for eavesdropping

[DBG]	bob_results       = {0: '0'}
	alice_results     = {0: '0'}

[DBG]	error_count       = 1
	len(indices_err1) = 1
	threshold         = 0.5

[ERR]	Eavesdropping (1) detected! Restarting QISAC protocol...



[STARTING QISAC PROTOCOL]



[NEW]	(1) Prepare list of EPR pairs and send one qubit from each to Bob

[DBG]	m      = 12
	m_msg  = 9
	m_err1 = 1
	m_err2 = 2

[DBG]	12 EPR pairs generated



[NEW]	(2) Check for eavesdropping

[DBG]	bob_results       = {2: '0'}
	alice_results     = {2: '1'}

[DBG]	error_count       = 0
	len(indices_err1) = 1
	threshold         = 0.5

[ ✓ ]	No eavesdropping detected. Continuing...

[DBG]	Removing eavesdrop detection circuits from circuits list



[NEW]	(3) Encode message and parameter estimate

[DBG]	Padded message    = 011101110
